In [ ]:
import os, glob
import ROOT
ROOT.gROOT.SetBatch(True)
ROOT.gStyle.SetOptStat(0)
ROOT.EnableImplicitMT()

# ---------------- simple inputs ----------------
TREE = "Events"
DIRS = [
    "/scratch-cbe/users/robert.schoefbeck/TT2lUnbinned/nanoTuples/TT2lUnbinned_v7/UL2016/dilep/TTLep_pow_CP5",
    # "/scratch-cbe/users/robert.schoefbeck/TT2lUnbinned/nanoTuples/TT2lUnbinned_v7/UL2016_preVFP/dilep/TTLep_pow_CP5",
    # "/scratch-cbe/users/robert.schoefbeck/TT2lUnbinned/nanoTuples/TT2lUnbinned_v7/UL2017/dilep/TTLep_pow_CP5",
    # "/scratch-cbe/users/robert.schoefbeck/TT2lUnbinned/nanoTuples/TT2lUnbinned_v7/UL2018/dilep/TTLep_pow_CP5",
]
TR_BRANCHES = [
    # fill with the top-reco variables you want, e.g.:
    "tr_Wminus_phi", "tr_ttbar_mass",  # add more tr_* here
]

XPROD_BINS = [5e-4, 1e-3, 2e-3, 5e-3, 1e-2]  # slices in x1*x2
OUTPNG, OUTPDF = "roberts_task_png", "roberts_task_pdf"
os.makedirs(OUTPNG, exist_ok=True)
os.makedirs(OUTPDF, exist_ok=True)

# ---------------- build file list ----------------
FILES = []
for d in DIRS:
    FILES.extend(sorted(glob.glob(os.path.join(d, "*.root"))))
assert FILES, "No ROOT files found. Check DIRS."

# ---------------- dataframe + cuts ----------------
df = ROOT.RDataFrame(TREE, FILES)\
       .Filter("tr_ttbar_mass > 500")\
       .Filter("abs(Generator_id1)==21 && abs(Generator_id2)==21")\
       .Define("x1", "Generator_x1")\
       .Define("x2", "Generator_x2")\
       .Define("xprod", "x1*x2")

# ---------------- helper to autoset range ----------------
def auto_range(dfr, col, nb=60):
    vmin = float(dfr.Min(col).GetValue())
    vmax = float(dfr.Max(col).GetValue())
    rng  = vmax - vmin
    if not (rng > 0): rng = 1.0
    lo, hi = vmin - 0.5*rng, vmax + 0.5*rng
    return nb, lo, hi

# ---------------- 1D: x1, x2, x1*x2 ----------------
for name in ["x1", "x2", "xprod"]:
    nb, lo, hi = auto_range(df, name, 80)
    h = df.Histo1D((f"h_{name}", f"{name};{name};Events", nb, lo, hi), name).GetValue()
    c = ROOT.TCanvas(f"c_{name}", "c", 900, 700); c.SetLogy(True)
    h.SetLineWidth(2); h.Draw("HIST"); c.Update()
    c.SaveAs(os.path.join(OUTPNG, f"{name}.png"))
    c.SaveAs(os.path.join(OUTPDF, f"{name}.pdf"))

# ---------------- 2D scatter: x2 vs x1 ----------------
nbx, xlo, xhi = auto_range(df, "x1", 120)
nby, ylo, yhi = auto_range(df, "x2", 120)
h2 = df.Histo2D(("h_x1x2", "x2 vs x1; x1; x2", nbx, xlo, xhi, nby, ylo, yhi), "x1", "x2").GetValue()
c2 = ROOT.TCanvas("c_x1x2", "c", 900, 800)
h2.Draw("COLZ"); c2.Update()
c2.SaveAs(os.path.join(OUTPNG, "x1_vs_x2.png"))
c2.SaveAs(os.path.join(OUTPDF, "x1_vs_x2.pdf"))

# ---------------- tr_* in slices of x1*x2 ----------------
for i in range(len(XPROD_BINS)-1):
    lo, hi = XPROD_BINS[i], XPROD_BINS[i+1]
    dfs = df.Filter(f"xprod >= {lo} && xprod < {hi}")
    tag = f"xprod_{lo:g}_{hi:g}".replace('.', 'p')
    for br in TR_BRANCHES:
        nb, rlo, rhi = auto_range(dfs, br, 80)
        h = dfs.Histo1D((f"h_{br}_{tag}", f"{br}  [{lo:g},{hi:g});{br};Events", nb, rlo, rhi), br).GetValue()
        c = ROOT.TCanvas(f"c_{br}_{tag}", "c", 900, 700); c.SetLogy(True)
        h.SetLineWidth(2); h.Draw("HIST"); c.Update()
        c.SaveAs(os.path.join(OUTPNG, f"{br}_{tag}.png"))
        c.SaveAs(os.path.join(OUTPDF, f"{br}_{tag}.pdf"))


: 

: 

: 